# MSDS 460 Assignment 1
Currency conversion problem using Linear Programming

In [83]:
# import all of the libraries we need
import numpy as np
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, LpContinuous
from matplotlib import pyplot as plt


### Problem Overview
We have a bucket of money in a few different currencies, and we're trying to maximize the amount of money 
we have after some exchanges, while satisfying a few constraints.

First, let's look at the conversion rates.

In [84]:
# currency we have (bid) rates are along the vertical axis,
# what we end up with (ask) is on the horizontal axis
# order of currencies is:
# USD, EUR, GBP, HKD, JPY
rates = np.array([[1, 1.01864, .064090, 7.7985, 118.55],
                 [.97240, 1, .62950, 7.65520, 116.41],
                 [1.5593, 1.5881, 1, 12.154, 184.97],
                 [.12812, .13040, .08210, 1, 15.10050],
                 [.00843, .00456, .00540, .06580, 1]])

# and what we currently have in millions, converted to USD
pot = np.array([2, 5, 1, 3, 30])
pot_conv = np.matmul(pot,rates)

print(f"Current holdings in USD: {pot_conv[0]:.2f} million")
print(f"Current holdings in EUR: {pot_conv[1]:.2f} million")
print(f"Current holdings in GBP: {pot_conv[2]:.2f} million")
print(f"Current holdings in HKD: {pot_conv[3]:.2f} million")
print(f"Current holdings in JPY: {pot_conv[4]:.2f} million")

Current holdings in USD: 9.06 million
Current holdings in EUR: 9.15 million
Current holdings in GBP: 4.68 million
Current holdings in HKD: 71.00 million
Current holdings in JPY: 1079.42 million


### Outlining the problem

We are wanting to increase our euro and yen holdings to 8 million EUR and 54 million JPY, while maintaining at least 250k in USD in each currency. 
For calculating the amount for each currency, I will calculate the values using the bid conversion, not the ask.
That means that I will need at least EUR*.9724 >= $250k, for example

So, that means that we need to have the following constraints (in millions)

<code>
USD >= .25;
.9724*EUR >= .25;
1.5593*GBP >= .25;
.12812*HKD >= .25;
.00843*JPY >= .25;


EUR >= 8;
JPY >= 54 </code>

### Problem 1
__Create a model for this problem and solve it__

So unless I'm misunderstanding this problem, what we have to do is set up a variable for each transaction, not each final quantity.
That means a variable for conversion from USD to GBP, one for USD to EUR, etc. We also need a term for any residual cash that isn't converted from a particular currency.

From there, we define each of our constraints using a combinations of these transactions. So, for USD we would have the initial constraint, which is

<code> 2M USD == USD2EUR + USD2GBP + USD2HKD + USD2JPY + USD2USD [residual, not converted] </code>

and also

<code> .25M USD <= USD2USD + .97240 * EUR2USD + 1.5593 GBP2USD + .12812 HKD2USD + .00843 JPY2USD </code>

In [85]:
# set up variables for each currency type
U2E = LpVariable("U2E", lowBound=0, upBound=2) # How many USD to Euroes. Can't be negative
U2G = LpVariable("U2G", lowBound=0, upBound=2) # "" to GBP
U2H = LpVariable("U2H", lowBound=0, upBound=2)
U2J = LpVariable("U2J", lowBound=0, upBound=2)
Ures = LpVariable("Ures", lowBound=0, upBound=2) # residual. Can I drop this term if I set the initial conditions to an inequality? Maybe...

E2U = LpVariable("E2U", lowBound=0, upBound=5) # EUR 2 USD
E2G = LpVariable("E2G", lowBound=0, upBound=5)
E2H = LpVariable("E2H", lowBound=0, upBound=5)
E2J = LpVariable("E2J", lowBound=0, upBound=5)
Eres = LpVariable("Eres", lowBound=0, upBound=5)

G2U = LpVariable("G2U", lowBound=0, upBound=1)
G2E = LpVariable("G2E", lowBound=0, upBound=1)
G2H = LpVariable("G2H", lowBound=0, upBound=1)
G2J = LpVariable("G2J", lowBound=0, upBound=1)
Gres = LpVariable("Gres", lowBound=0, upBound=1)

H2U = LpVariable("H2U", lowBound=0, upBound=3)
H2E = LpVariable("H2E", lowBound=0, upBound=3)
H2G = LpVariable("H2G", lowBound=0, upBound=3)
H2J = LpVariable("H2J", lowBound=0, upBound=3)
Hres = LpVariable("Hres", lowBound=0, upBound=3)

J2U = LpVariable("J2U", lowBound=0, upBound=30)
J2E = LpVariable("J2E", lowBound=0, upBound=30)
J2G = LpVariable("J2G", lowBound=0, upBound=30)
J2H = LpVariable("J2H", lowBound=0, upBound=30)
Jres = LpVariable("Jres", lowBound=0, upBound=30)

# to make the objective functions easier, we'll create a variable
# for the final amount in each currency
USDf = LpVariable("USDf")
EURf = LpVariable("EURf")
GBPf = LpVariable("GBPf")
HKDf = LpVariable("HKDf")
JPYf = LpVariable("JPYf")

# now the constraints:
prob = LpProblem('MoneyConversion', LpMaximize)
# Initial values
prob += Ures + U2E + U2G + U2H + U2J == 2
prob += Eres + E2U + E2G + E2H + E2J == 5
prob += Gres + G2U + G2E + G2H + G2J == 1
prob += Hres + H2U + H2E + H2G + H2J == 3
prob += Jres + J2U + J2E + J2G + J2H == 30

# The total amounts converted must be less than or equal to the final conversion values
prob += Ures +         .9724*E2U +   1.5593*G2U +  .12812*H2U +   .00843*J2U -  USDf == 0
prob += 1.01864*U2E +  Eres +        1.5881*G2E +  .1304*H2E +    .00856*J2E -  EURf == 0# conversion of .25 to Euro using Bid rate, not Ask
prob += .6409*U2G +    .6295*E2G +   Gres +         .08210*H2G +  .0054*J2G  -  GBPf == 0 # ""
prob += 7.7985*U2H +   7.6552*E2H +  12.154*G2H +   Hres +        .0658*J2H  -  HKDf == 0
prob += 118.55*U2J +   116.41*E2J +  184.97*G2J +   15.1005*H2J +  Jres      -  JPYf == 0


# minimums for the final values
prob += USDf >= .25
prob += EURf >= .25/.9724  # using the Bid values, not the ask
prob += GBPf >= .25/1.5593
prob += HKDf >= .25/.12812
prob += JPYf >= .25/.00843

# Need at least 8M GBP and 24M JPY
prob +=  EURf >= 8
prob +=  JPYf >= 54 

# and the objective function: how much do we have of each, in USD?
prob += USDf + .9724*EURf + 1.5593*GBPf + .12812*HKDf + .00843*JPYf 

In [86]:

# solve it, and print everything really pretty
prob.solve()


1

In [87]:
print(f"We want (in millions): {USDf.value()} USD, {EURf.value()} EUR, {GBPf.value()} GBP, {HKDf.value()} HKD, {JPYf.value()} JPY")
print(f"This gives us a total of {prob.objective.value()} million USD")


We want (in millions): 0.29608381 USD, 8.0 EUR, 0.16032835 GBP, 1.9512957 HKD, 54.0 JPY
This gives us a total of 9.030503811239 million USD


In [88]:
# equivalents of each in USD
print(EURf.value()*.9724)
print(GBPf.value()*1.5593)
print(HKDf.value()*.12812)
print(JPYf.value()*.00843)

7.7792
0.249999996155
0.250000005084
0.45522


### Trading plan (question 2)
Since we created variables for each trade, we just need to print out all of the variables. Isn't that nice?

In [89]:
for var in prob.variables():
    print(f"{var}: {var.value()}")

E2G: 0.0
E2H: 0.0
E2J: 0.0
E2U: 0.0
EURf: 8.0
Eres: 5.0
G2E: 0.60620868
G2H: 0.0
G2J: 0.23346297
G2U: 0.0
GBPf: 0.16032835
Gres: 0.16032835
H2E: 0.0
H2G: 0.0
H2J: 0.0
H2U: 2.3109882
HKDf: 1.9512957
Hres: 0.68901178
J2E: 0.0
J2G: 0.0
J2H: 19.183646
J2U: 0.0
JPYf: 54.0
Jres: 10.816354
U2E: 2.0
U2G: 0.0
U2H: 0.0
U2J: 0.0
USDf: 0.29608381
Ures: 0.0


### Transaction Costs (Question 3)

Subtract post-transaction costs from initial valuation in USD

In [90]:
print(f"Transaction costs: {pot_conv[0] - prob.objective.value()}M USD")

Transaction costs: 0.028056188760999135M USD


### Changing the minimum holdings (Question 4)

What happens if we change the minimum holdings from 250k to 50k?

In [96]:
# set up variables for each currency type
U2E = LpVariable("U2E", lowBound=0, upBound=2) # How many USD to Euroes. Can't be negative
U2G = LpVariable("U2G", lowBound=0, upBound=2) # "" to GBP
U2H = LpVariable("U2H", lowBound=0, upBound=2)
U2J = LpVariable("U2J", lowBound=0, upBound=2)
Ures = LpVariable("Ures", lowBound=0, upBound=2) # residual. Can I drop this term if I set the initial conditions to an inequality? Maybe...

E2U = LpVariable("E2U", lowBound=0, upBound=5) # EUR 2 USD
E2G = LpVariable("E2G", lowBound=0, upBound=5)
E2H = LpVariable("E2H", lowBound=0, upBound=5)
E2J = LpVariable("E2J", lowBound=0, upBound=5)
Eres = LpVariable("Eres", lowBound=0, upBound=5)

G2U = LpVariable("G2U", lowBound=0, upBound=1)
G2E = LpVariable("G2E", lowBound=0, upBound=1)
G2H = LpVariable("G2H", lowBound=0, upBound=1)
G2J = LpVariable("G2J", lowBound=0, upBound=1)
Gres = LpVariable("Gres", lowBound=0, upBound=1)

H2U = LpVariable("H2U", lowBound=0, upBound=3)
H2E = LpVariable("H2E", lowBound=0, upBound=3)
H2G = LpVariable("H2G", lowBound=0, upBound=3)
H2J = LpVariable("H2J", lowBound=0, upBound=3)
Hres = LpVariable("Hres", lowBound=0, upBound=3)

J2U = LpVariable("J2U", lowBound=0, upBound=30)
J2E = LpVariable("J2E", lowBound=0, upBound=30)
J2G = LpVariable("J2G", lowBound=0, upBound=30)
J2H = LpVariable("J2H", lowBound=0, upBound=30)
Jres = LpVariable("Jres", lowBound=0, upBound=30)

# to make the objective functions easier, we'll create a variable
# for the final amount in each currency
USDf = LpVariable("USDf")
EURf = LpVariable("EURf")
GBPf = LpVariable("GBPf")
HKDf = LpVariable("HKDf")
JPYf = LpVariable("JPYf")

# now the constraints:
prob_4 = LpProblem('MoneyConversion', LpMaximize)
# Initial values
prob_4 += Ures + U2E + U2G + U2H + U2J == 2
prob_4 += Eres + E2U + E2G + E2H + E2J == 5
prob_4 += Gres + G2U + G2E + G2H + G2J == 1
prob_4 += Hres + H2U + H2E + H2G + H2J == 3
prob_4 += Jres + J2U + J2E + J2G + J2H == 30

# The total amounts converted must be less than or equal to the final conversion values
prob_4 += Ures +         .9724*E2U +   1.5593*G2U +  .12812*H2U +   .00843*J2U -  USDf == 0
prob_4 += 1.01864*U2E +  Eres +        1.5881*G2E +  .1304*H2E +    .00856*J2E -  EURf == 0# conversion of .25 to Euro using Bid rate, not Ask
prob_4 += .6409*U2G +    .6295*E2G +   Gres +         .08210*H2G +  .0054*J2G  -  GBPf == 0 # ""
prob_4 += 7.7985*U2H +   7.6552*E2H +  12.154*G2H +   Hres +        .0658*J2H  -  HKDf == 0
prob_4 += 118.55*U2J +   116.41*E2J +  184.97*G2J +   15.1005*H2J +  Jres      -  JPYf == 0


# minimums for the final values
prob_4 += USDf >= .05
prob_4 += EURf >= .05/.9724  # using the Bid values, not the ask
prob_4 += GBPf >= .05/1.5593
prob_4 += HKDf >= .05/.12812
prob_4 += JPYf >= .05/.00843

# Need at least 8M GBP and 24M JPY
prob_4 +=  EURf >= 8
prob_4 +=  JPYf >= 54 

# and the objective function: how much do we have of each, in USD?
prob_4 += USDf + .9724*EURf + 1.5593*GBPf + .12812*HKDf + .00843*JPYf 

prob_4.solve()

1

In [100]:
# print out the values
print(f"We want (in millions): {USDf.value()} USD, {EURf.value()} EUR, {GBPf.value()} GBP, {HKDf.value()} HKD, {JPYf.value()} JPY")
print(f"This gives us a total of {prob.objective.value()} million USD")

print(f"This gives us an objective cost of {pot_conv[0]-prob_4.objective.value()}")

We want (in millions): 0.05 USD, 8.0 EUR, 0.10185209 GBP, 4.5837409 HKD, 54.0 JPY
This gives us a total of 9.030506848045 million USD
This gives us an objective cost of 0.028053151954999578


In [98]:
# equivalents in USD -- using the bid to USD
print(EURf.value()*.9724)
print(GBPf.value()*1.5593)
print(HKDf.value()*.12812)
print(JPYf.value()*.00843)

7.7792
0.158817963937
0.587268884108
0.45522


In [99]:
for var in prob_4.variables():
    print(f"{var}:{var.value()}")

E2G:0.0
E2H:0.0
E2J:0.0
E2U:0.0
EURf:8.0
Eres:5.0
G2E:0.60620868
G2H:0.0
G2J:0.29193923
G2U:0.0
GBPf:0.10185209
Gres:0.10185209
H2E:0.0
H2G:0.0
H2J:0.0
H2U:0.39025913
HKDf:4.5837409
Hres:2.6097409
J2E:0.0
J2G:0.0
J2H:30.0
J2U:0.0
JPYf:54.0
Jres:0.0
U2E:2.0
U2G:0.0
U2H:0.0
U2J:0.0
USDf:0.05
Ures:0.0


### Exchange rate change (Question 5)

Changing the USD to GBP rate from .6409 to .6414

In [105]:
# set up variables for each currency type
U2E = LpVariable("U2E", lowBound=0, upBound=2) # How many USD to Euroes. Can't be negative
U2G = LpVariable("U2G", lowBound=0, upBound=2) # "" to GBP
U2H = LpVariable("U2H", lowBound=0, upBound=2)
U2J = LpVariable("U2J", lowBound=0, upBound=2)
Ures = LpVariable("Ures", lowBound=0, upBound=2) # residual. Can I drop this term if I set the initial conditions to an inequality? Maybe...

E2U = LpVariable("E2U", lowBound=0, upBound=5) # EUR 2 USD
E2G = LpVariable("E2G", lowBound=0, upBound=5)
E2H = LpVariable("E2H", lowBound=0, upBound=5)
E2J = LpVariable("E2J", lowBound=0, upBound=5)
Eres = LpVariable("Eres", lowBound=0, upBound=5)

G2U = LpVariable("G2U", lowBound=0, upBound=1)
G2E = LpVariable("G2E", lowBound=0, upBound=1)
G2H = LpVariable("G2H", lowBound=0, upBound=1)
G2J = LpVariable("G2J", lowBound=0, upBound=1)
Gres = LpVariable("Gres", lowBound=0, upBound=1)

H2U = LpVariable("H2U", lowBound=0, upBound=3)
H2E = LpVariable("H2E", lowBound=0, upBound=3)
H2G = LpVariable("H2G", lowBound=0, upBound=3)
H2J = LpVariable("H2J", lowBound=0, upBound=3)
Hres = LpVariable("Hres", lowBound=0, upBound=3)

J2U = LpVariable("J2U", lowBound=0, upBound=30)
J2E = LpVariable("J2E", lowBound=0, upBound=30)
J2G = LpVariable("J2G", lowBound=0, upBound=30)
J2H = LpVariable("J2H", lowBound=0, upBound=30)
Jres = LpVariable("Jres", lowBound=0, upBound=30)

# to make the objective functions easier, we'll create a variable
# for the final amount in each currency
USDf = LpVariable("USDf")
EURf = LpVariable("EURf")
GBPf = LpVariable("GBPf")
HKDf = LpVariable("HKDf")
JPYf = LpVariable("JPYf")

# now the constraints:
prob_5 = LpProblem('MoneyConversion', LpMaximize)
# Initial values
prob_5 += Ures + U2E + U2G + U2H + U2J == 2
prob_5 += Eres + E2U + E2G + E2H + E2J == 5
prob_5 += Gres + G2U + G2E + G2H + G2J == 1
prob_5 += Hres + H2U + H2E + H2G + H2J == 3
prob_5 += Jres + J2U + J2E + J2G + J2H == 30

# The total amounts converted must be less than or equal to the final conversion values
prob_5 += Ures +         .9724*E2U +   1.5593*G2U +  .12812*H2U +   .00843*J2U -  USDf == 0
prob_5 += 1.01864*U2E +  Eres +        1.5881*G2E +  .1304*H2E +    .00856*J2E -  EURf == 0# conversion of .25 to Euro using Bid rate, not Ask
prob_5 += .6414*U2G +    .6295*E2G +   Gres +         .08210*H2G +  .0054*J2G  -  GBPf == 0 # ""
prob_5 += 7.7985*U2H +   7.6552*E2H +  12.154*G2H +   Hres +        .0658*J2H  -  HKDf == 0
prob_5 += 118.55*U2J +   116.41*E2J +  184.97*G2J +   15.1005*H2J +  Jres      -  JPYf == 0


# minimums for the final values
prob_5 += USDf >= .25
prob_5 += EURf >= .25/.9724  # using the Bid values, not the ask
prob_5 += GBPf >= .25/1.5593
prob_5 += HKDf >= .25/.12812
prob_5 += JPYf >= .25/.00843

# Need at least 8M GBP and 24M JPY
prob_5 +=  EURf >= 8
prob_5 +=  JPYf >= 54 

# and the objective function: how much do we have of each, in USD?
prob_5 += USDf + .9724*EURf + 1.5593*GBPf + .12812*HKDf + .00843*JPYf 

prob_5.solve()

1

In [106]:
# print out the values
print(f"We want (in millions): {USDf.value()} USD, {EURf.value()} EUR, {GBPf.value()} GBP, {HKDf.value()} HKD, {JPYf.value()} JPY")
print(f"This gives us a total of {prob.objective.value()} million USD")

print(f"This gives us an objective cost of {pot_conv[0]-prob_5.objective.value()}")

We want (in millions): 0.29608381 USD, 8.0 EUR, 0.16032835 GBP, 1.9512957 HKD, 54.0 JPY
This gives us a total of 9.030506848045 million USD
This gives us an objective cost of 0.028056188760999135


In [108]:
for var in prob_5.variables():
    print(f"{var}:{var.value()}")

E2G:0.0
E2H:0.0
E2J:0.0
E2U:0.0
EURf:8.0
Eres:5.0
G2E:0.60620868
G2H:0.0
G2J:0.23346297
G2U:0.0
GBPf:0.16032835
Gres:0.16032835
H2E:0.0
H2G:0.0
H2J:0.0
H2U:2.3109882
HKDf:1.9512957
Hres:0.68901178
J2E:0.0
J2G:0.0
J2H:19.183646
J2U:0.0
JPYf:54.0
Jres:10.816354
U2E:2.0
U2G:0.0
U2H:0.0
U2J:0.0
USDf:0.29608381
Ures:0.0
